In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set image dimensions
IMG_SIZE = (128, 128)

# Print the current working directory to ensure we're working in the right path
print(f"Current working directory: {os.getcwd()}")

# Function to load and preprocess images
def load_images_from_directory(directory):
    images = []
    labels = []
    for label_dir in os.listdir(directory):
        label_path = os.path.join(directory, label_dir)
        if not os.path.isdir(label_path):
            continue
        label = 1 if label_dir.lower() == "pneumonia" else 0  # 1 for pneumonia, 0 for normal
        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            try:
                # Load, resize, convert to grayscale, and normalize
                img = Image.open(img_path).resize(IMG_SIZE).convert('L')
                img_array = np.array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Could not process {img_file}: {e}")
    return np.array(images), np.array(labels)

# Load training and testing images
train_dir = 'path/to/train'
test_dir = 'path/to/test'

X_train, y_train = load_images_from_directory(train_dir)
X_test, y_test = load_images_from_directory(test_dir)


# Reshape images to 1D vectors
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)


# Initialize KNN classifier with chosen k value
k = 5
knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')  # Euclidean distance is a standard choice

# Fit the model (storing training data)
knn.fit(X_train, y_train)

# Predict on test data
y_pred = knn.predict(X_test)


# Accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
report = classification_report(y_test, y_pred, target_names=["Normal", "Pneumonia"])
print("Classification Report:")
print(report)


# Plot confusion matrix
plt.figure(figsize=(6, 6))
plt.imshow(conf_matrix, cmap="Reds", interpolation="nearest")
plt.colorbar()
plt.title("Confusion Matrix")
plt.xticks([0, 1], ["Normal", "Pneumonia"])
plt.yticks([0, 1], ["Normal", "Pneumonia"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()


# Calculate ROC Curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Initialize KNN classifier with chosen k value
k = 7
knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')  # Euclidean distance is a standard choice

# Perform cross-validation
cv_scores = cross_val_score(knn, X_train, y_train, cv=5)  # 5-fold cross-validation
# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())
print("Standard deviation of cross-validation scores:", cv_scores.std())

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

# Set image dimensions
IMG_SIZE = (128, 128)
train_dir = "/Users/nidhikukunooru/Downloads/chest_xray/train"
test_dir = "/Users/nidhikukunooru/Downloads/chest_xray/test"

# Print the current working directory to ensure we're working in the right path
print(f"Current working directory: {os.getcwd()}")

# Function to load and preprocess images
def load_images_from_directory(directory):
    images = []
    labels = []
    supported_extensions = ('.png', '.jpg', '.jpeg')

    for label_dir in os.listdir(directory):
        label_path = os.path.join(directory, label_dir)
        if not os.path.isdir(label_path):
            continue

        label = 1 if label_dir.lower() == "pneumonia" else 0  # 1 for pneumonia, 0 for normal

        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)

            # Only process supported image files and skip hidden files
            if img_file.lower().endswith(supported_extensions) and not img_file.startswith('.'):
                try:
                    # Load, resize, convert to grayscale, and normalize
                    img = Image.open(img_path).resize(IMG_SIZE).convert('L')
                    img_array = np.array(img) / 255.0  # Normalize
                    images.append(img_array)
                    labels.append(label)
                except Exception as e:
                    print(f"Could not process {img_file}: {e}")
            else:
                print(f"Skipping unsupported file: {img_file}")

    return np.array(images), np.array(labels)

# Load and preprocess all images in the directory
X_train, y_train = load_images_from_directory(train_dir)
X_test, y_test = load_images_from_directory(test_dir)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=42)


# Reshape images to 1D vectors
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Initialize KNN classifier with chosen k value
k = 7
knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')  # Euclidean distance is a standard choice

# Perform cross-validation
cv_scores = cross_val_score(knn, X_train, y_train, cv=5)  # 5-fold cross-validation
# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())
print("Standard deviation of cross-validation scores:", cv_scores.std())

# Fit the model (storing training data)
knn.fit(X_train, y_train)

# Predict on test data
y_pred = knn.predict(X_test)

# Get predicted probabilities for the positive class (pneumonia)
y_scores = knn.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
report = classification_report(y_test, y_pred, target_names=["Normal", "Pneumonia"])
print("Classification Report:")
print(report)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
plt.imshow(conf_matrix, cmap="Reds", interpolation="nearest")
plt.colorbar()
plt.title("Confusion Matrix")
plt.xticks([0, 1], ["Normal", "Pneumonia"])
plt.yticks([0, 1], ["Normal", "Pneumonia"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Calculate ROC Curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()